#  Neural Networks: Regression on House Pricing Dataset
We consider a reduced version of a dataset containing house sale prices for King County, which includes Seattle. It includes homes sold between May 2014 and May 2015.

https://www.kaggle.com/harlfoxem/housesalesprediction

For each house we know 18 house features (e.g., number of bedrooms, number of bathrooms, etc.) plus its price, that is what we would like to predict.

In [3]:
#put here your ``numero di matricola''
numero_di_matricola = 2095665

## Insert your ID number ("numero di matricola") below

In [1]:
#put here your ``numero di matricola''
numero_di_matricola = 2095665 # COMPLETE

In [2]:
#import all packages needed
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Load the data, remove data samples/points with missing values (NaN) and take a look at them.

In [4]:
#load the data
df = pd.read_csv('kc_house_data.csv', sep = ',')

#remove the data samples with missing values (NaN)
df = df.dropna() 

df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,3.164000e+03,3.164000e+03,3164.000000,3164.000000,3164.000000,3.164000e+03,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000
mean,4.645240e+09,5.354358e+05,3.381163,2.071903,2070.027813,1.525054e+04,1.434893,0.009798,0.244311,3.459229,7.615676,1761.252212,308.775601,1967.489254,94.668774,98077.125158,47.557868,-122.212337,1982.544564,13176.302465
std,2.854203e+09,3.809004e+05,0.895472,0.768212,920.251879,4.254457e+04,0.507792,0.098513,0.776298,0.682592,1.166324,815.934864,458.977904,28.095275,424.439427,54.172937,0.140789,0.139577,686.256670,25413.180755
min,1.000102e+06,7.500000e+04,0.000000,0.000000,380.000000,6.490000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,380.000000,0.000000,1900.000000,0.000000,98001.000000,47.177500,-122.514000,620.000000,660.000000
25%,2.199775e+09,3.150000e+05,3.000000,1.500000,1430.000000,5.453750e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1950.000000,0.000000,98032.000000,47.459575,-122.324250,1480.000000,5429.500000
50%,4.027701e+09,4.450000e+05,3.000000,2.000000,1910.000000,8.000000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1545.000000,0.000000,1969.000000,0.000000,98059.000000,47.572500,-122.226000,1830.000000,7873.000000
75%,7.358175e+09,6.402500e+05,4.000000,2.500000,2500.000000,1.122250e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2150.000000,600.000000,1990.000000,0.000000,98117.000000,47.680250,-122.124000,2360.000000,10408.250000
max,9.839301e+09,5.350000e+06,8.000000,6.000000,8010.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,12.000000,6720.000000,2620.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,5790.000000,425581.000000


Extract input and output data. We want to predict the price by using features other than id as input.

In [5]:
Data = df.values
# m = number of input samples
m = Data.shape[0]
print("Amount of data:",m)
Y = Data[:m,2]
X = Data[:m,3:]

Amount of data: 3164


## Data Pre-Processing

We split the data into 3 parts: one will be used for training and choosing the parameters, one for choosing among different models, and one for testing. The part for training and choosing the parameters will consist of $2/3$ of all samples, the one for choosing among different models will consist of $1/6$ of all samples, while the other part consists of the remaining $1/6$-th of all samples.

In [6]:
# Split data into train (2/3 of samples), validation (1/6 of samples), and test data (the rest)
m_train = int(2./3.*m)
m_val = int((m-m_train)/2.)
m_test = m - m_train - m_val
print("Amount of data for training and deciding parameters:",m_train)
print("Amount of data for validation (choosing among different models):",m_val)
print("Amount of data for test:",m_test)
from sklearn.model_selection import train_test_split

Xtrain_and_val, Xtest, Ytrain_and_val, Ytest = train_test_split(X, Y, test_size=m_test/m, random_state=numero_di_matricola)
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtrain_and_val, Ytrain_and_val, test_size=m_val/(m_train+m_val), random_state=numero_di_matricola)

Amount of data for training and deciding parameters: 2109
Amount of data for validation (choosing among different models): 527
Amount of data for test: 528


Let's standardize the data.

In [7]:
# Data pre-processing
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(Xtrain)
Xtrain_scaled = scaler.transform(Xtrain)
Xtrain_and_val_scaled = scaler.transform(Xtrain_and_val)
Xval_scaled = scaler.transform(Xval)
Xtest_scaled = scaler.transform(Xtest)

## Neural Networks
Let's start by learning a simple neural network with 1 hidden node.
Note: we are going to use the input parameter solver='lbfgs' and random_state=numero_di_matricola to fix the random seed (so results are reproducible).

In [11]:
#let's load the MLPRegressor

#COMPLETE
from sklearn.neural_network import MLPRegressor

#let's define the model
#COMPLETE
mlp = MLPRegressor(hidden_layer_sizes = (1, ),solver = 'lbfgs', random_state = numero_di_matricola)

#let's learn the model on training data
#COMPLETE
mlp.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
#COMPLETE
print("Training error: ",1. -mlp.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
#COMPLETE
print("Test error: ",1. -mlp.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
#COMPLETE
print(mlp.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
#COMPLETE
print(mlp.intercepts_)


Training error:  0.2824211590390904
Test error:  0.3086976163229931
[array([[-22.09284651],
       [ 25.83198242],
       [ 58.29430386],
       [  5.99114412],
       [  0.97414478],
       [ 42.24931325],
       [ 35.91335419],
       [  4.01617389],
       [ 79.00865986],
       [ 54.12517556],
       [ 20.58044315],
       [-61.53301516],
       [  4.88152448],
       [-21.68810827],
       [ 57.37723108],
       [-18.35295007],
       [ 19.14160751],
       [ -9.64659594]]), array([[1382.30082149]])]
[array([387.89958707]), array([3517.42938062])]


## Neural Networks vs Linear Models

Let's learn a linear model on the other same data and compare the results with the simple NN above.

In [13]:
#COMPLETE
from sklearn import linear_model

LR = linear_model.LinearRegression()

LR.fit(Xtrain_scaled, Ytrain)

print("Training error: ",1. -LR.score(Xtrain_scaled, Ytrain))

print("Test error: ",1. -LR.score(Xval_scaled, Yval))

print(LR.coef_)

print(LR.intercept_)


Training error:  0.28400284161011513
Test error:  0.31253928777675377
[-32409.77990598  33403.71470664  81429.33410886   7391.69685349
   1533.54604969  58056.9748205   49793.62053169   5886.53376008
 104293.52034659  75387.95389241  29332.54735303 -80689.38903083
   7898.73436135 -29520.19760068  78278.82683066 -26077.98203206
  27563.28326827 -10877.15519539]
541490.9881459652


Is there a way to make a NN network learn a linear model?

Let's first check what is the loss used by MLPRegressor...

In [15]:
#let's write the code to learn a linear model with NN: how? 

#COMPLETE
#let's load the MLPRegressor

#let's define the model
#COMPLETE
mlp_lr = MLPRegressor(hidden_layer_sizes = (1, ),solver = 'lbfgs', random_state = numero_di_matricola)

#let's learn the model on training data
#COMPLETE
mlp_lr.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
#COMPLETE
print("Training error: ",1. -mlp_lr.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
#COMPLETE
print("Test error: ",1. -mlp_lr.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
#COMPLETE
print(mlp_lr.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
#COMPLETE
print(mlp_lr.intercepts_)


Training error:  0.2824211590390904
Test error:  0.3086976163229931
[array([[-22.09284651],
       [ 25.83198242],
       [ 58.29430386],
       [  5.99114412],
       [  0.97414478],
       [ 42.24931325],
       [ 35.91335419],
       [  4.01617389],
       [ 79.00865986],
       [ 54.12517556],
       [ 20.58044315],
       [-61.53301516],
       [  4.88152448],
       [-21.68810827],
       [ 57.37723108],
       [-18.35295007],
       [ 19.14160751],
       [ -9.64659594]]), array([[1382.30082149]])]
[array([387.89958707]), array([3517.42938062])]


Note that there is an $\ell_2$ regularization term in MLPRegressor. What about making it smaller?

In [ ]:
#COMPLETE
mlp_lr_noreg = MLPRegressor(hidden_layer_sizes = (1, ),solver = 'lbfgs', random_state = numero_di_matricola, activation = 'identity', alpha =)

#let's learn the model on training data
#COMPLETE
mlp_lr_noreg.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
#COMPLETE
print("Training error: ",1. -mlp_lr_noreg.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
#COMPLETE
print("Test error: ",1. -mlp_lr_noreg.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
#COMPLETE
print(mlp_lr_noreg.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
#COMPLETE
print(mlp_lr_noreg.intercepts_)


## More Complex NNs

Let's try more complex NN, for example increasing the number of nodes in the only hidden layer, or increasing the number of hidden layers.

Let's build a NN with 2 nodes in the only hidden layer

In [ ]:
#let's build a NN with 2 nodes in the only hidden layer

#COMPLETE


Let's build a NN with 5 nodes in the only hidden layer

In [ ]:
#let's build a NN with 5 nodes in the only hidden layer

#COMPLETE

Note that with a smaller number of iterations we had a larger error on training set but a smaller error on validation data -> "early stopping is a form of regularization"

Let's build a NN with 10 nodes in the only hidden layer

In [ ]:
#let's build a NN with 10 nodes in the only hidden layer

#COMPLETE

Let's build a NN with 100 nodes in the only hidden layer. Note that this is the default!

In [ ]:
#let's build a NN with 100 nodes in the only hidden layer

#COMPLETE

Let's try 2 layers, 1 node each

In [ ]:
#let's build a NN with 2 hidden layers, 1 node each

#COMPLETE

Let's try 2 layers, 2 nodes each

In [ ]:
#let's build a NN with 2 layers, 2 nodes each

#COMPLETE

Let's try 2 layers, 10 nodes each

In [ ]:
#let's build a NN with 2 layers, 10 nodes each

#COMPLETE

Let's try 2 layers, 100 nodes each

In [ ]:
#let's build a NN with 2 layers, 100 nodes each

#COMPLETE

So it seems that 1 layer (and default number of iterations) works best for this dataset. Let's try 5-fold cross-validation with number of nodes in the hidden layer between 1 and 20.
Note that we use train and validation data together, since we are doing cross-validation.

In [ ]:
from sklearn.model_selection import GridSearchCV

#COMPLETE

Now let's check what is the best parameter, and compare the best NNs with the linear model (learned on train and validation) on test data.

In [ ]:
#let's print the best model according to grid search
#COMPLETE

#let's print the error 1-R^2 for the best model
#COMPLETE

Let compare the error of the best NN on train and validation and on test data.

In [ ]:
#COMPLETE

Now let's learn the linear model on train and validation, and get error (1-R^2) on train and validation and on test data.

In [ ]:
#COMPLETE

Note: MLPRegressor has several other parameters!